In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

import warnings
warnings.filterwarnings("ignore")


In [2]:
df=pd.read_csv('/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/tdcsfog/76c215b221.csv')
df.head()


,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking
0,0,-8.514709,0.976285,-4.684719,0,0,0
1,1,-8.519063,0.979048,-4.681319,0,0,0
2,2,-8.516622,0.976981,-4.674121,0,0,0
3,3,-8.514855,0.978317,-4.638024,0,0,0
4,4,-8.517053,0.976239,-4.633303,0,0,0


In [3]:
tdsfog_meta =pd.read_csv('/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/tdcsfog_metadata.csv')

master_path ='/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/tdcsfog'

tdsfog_meta.shape
dfs=[]
master_df=pd.DataFrame()

for index, row in tdsfog_meta.iterrows():
    path = os.path.join(master_path, row['Id']+'.csv')
    df=pd.read_csv(path)
    df=df.dropna()
    dfs.append(df) 
    
master_df = pd.concat(dfs, axis=0, ignore_index=True)

In [4]:
def create_windows(df, feature_cols, target_cols, window_size):
    X, y = [], []
    data = df[feature_cols + target_cols].values
    for i in range(len(data) - window_size):
        X.append(data[i:i + window_size, :-len(target_cols)].flatten())
        y.append(data[i + window_size, -len(target_cols):])
    return X, y

In [5]:
X_all, y_all = [], []

for index, row in tdsfog_meta.iterrows():
    path = os.path.join(master_path, row['Id'] + '.csv')
    df = pd.read_csv(path)
    df = df.dropna()
    df = df.sort_values('Time')  # or whatever your time column is
    X, y = create_windows(df, feature_cols=['AccV', 'Time', 'AccAP', 'AccML'], target_cols= ['Walking','Turn','StartHesitation'], window_size=5)
    X_all.extend(X)
    y_all.extend(y)


In [9]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_all, y_all, test_size=0.8, shuffle=False)  # time-based, so no shuffle


In [10]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [11]:
print("Validation Accuracy:", model.score(X_train, y_train))


Validation Accuracy: 0.9999985832694034


In [12]:
print("Validation Accuracy:", model.score(X_val, y_val))


Validation Accuracy: 0.7372043594201749


In [ ]:
print(train_data.nunique())


In [ ]:

fig = go.Figure()

fig.add_trace(go.Scatter(x=df['Time'], y=df['AccV'], mode='lines', name='AccV'))
fig.add_trace(go.Scatter(x=df['Time'], y=df['AccML'], mode='lines', name='AccML'))
fig.add_trace(go.Scatter(x=df['Time'], y=df['AccAP'], mode='lines', name='AccAP'))

fig.update_layout(
    title='',
    xaxis_title='Time',
    yaxis_title='m/s^2'
)


In [ ]:

tdcsfog_metadata=pd.read_csv('/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/tdcsfog_metadata.csv')

tdcsfog_metadata.head()

In [ ]:


dataset_path = r'/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/tdcsfog/' 
extracted_features = []

for index, row in tqdm(tdcsfog_metadata.iterrows(), total=len(tdcsfog_metadata)):
    file_name = os.path.join(dataset_path, str(row["Id"]) + '.csv')
    
    # Load the full CSV file (time-series data)
    #if not os.path.isfile(file_name):
    #   continue  # skip if file not found

    df = pd.read_csv(file_name)
    
    # Extract features and labels over time
    features = df[["Time", "AccAP", "AccML"]].values        # shape (T, 3)
    labels = df[["Walking", "Turn", "StartHesitation"]].values  # shape (T, 3)

    extracted_features.append([features, labels])


In [ ]:
extracted_features